In [1]:
import pandas as pd
import numpy as np
import time
from pathlib import Path
import os
import datetime
from datetime import datetime, date, timedelta
import openpyxl

In [2]:
file_path = Path.home() / 'leave_project_2.1'

In [3]:
leave_df = pd.read_excel(file_path / '자료수정.xlsx', sheet_name='시트1', parse_dates=['부재 시작일은 언제입니까?', '부재 종료일은 언제입니까? (2일 이상의 부재인 경우만 기재)'])
leave_df

,이름,부재 일수를 적어주세요.(업무일 기준),부재 시작일은 언제입니까?,부재 종료일은 언제입니까? (2일 이상의 부재인 경우만 기재),부재 사유는 무엇입니까?
0,조윤아,5일,2020-01-20,2020-02-03,연차(일반)
1,이준우,1일,2020-01-02,NaT,연차(일반)
2,이준우,0.5일(오후),2020-01-03,NaT,연차(일반)
3,이준우,0.5일(오후),2020-01-08,NaT,연차(일반)
4,조윤아,1일,2020-01-03,NaT,연차(일반)
...,...,...,...,...,...
5030,장준연,0.5일(오전),2022-09-16,NaT,연차(병가)
5031,정예후,1일,2022-09-19,NaT,연차(일반)
5032,임은현,0.5일(오후),2022-09-16,NaT,연차(병가)
5033,강주윤,1일,2022-09-23,NaT,연차(일반)


In [4]:
info_df = pd.read_excel(file_path / '자료수정.xlsx', sheet_name='시트2', parse_dates=['시작일', '종료일'])
info_df

,구분,성명,소속,시작일,종료일,근속월수
0,퇴사(정규직),정윤호,운영팀,2012-06-24,2021-10-31,112.21
1,정규직,김채진,영업/마케팅팀,2012-08-25,2022-09-17,120.73
2,정규직,장민우,연구개발팀,2012-08-25,2022-09-17,120.73
3,정규직,윤윤원,연구개발팀,2013-07-29,2022-09-17,109.65
4,퇴사(정규직),윤준윤,운영팀,2015-03-11,2022-07-31,88.67
...,...,...,...,...,...,...
101,퇴사(기타),정지원,운영팀,2016-03-07,2016-06-13,3.21
102,퇴사(기타),박주진,영업/마케팅팀,2018-03-08,2018-06-30,3.75
103,퇴사(기타),윤준우,영업/마케팅팀,2019-03-11,2019-06-17,3.22
104,퇴사(기타),조도우,운영팀,2019-03-11,2019-05-31,2.66


In [5]:
leave_rename_cols = {'부재 일수를 적어주세요.(업무일 기준)':'부재일수', '부재 시작일은 언제입니까?':'부재시작일',
       '부재 종료일은 언제입니까? (2일 이상의 부재인 경우만 기재)':'부재종료일', '부재 사유는 무엇입니까?':'부재사유'}

In [6]:
leave_df = (leave_df
.rename(leave_rename_cols, axis=1)
# np.where보다는 fillna() 메서드를 이용하는 것이 훨씬 깔끔
.assign(부재종료일=lambda df: df['부재종료일'].fillna(df['부재시작일']))
.assign(반차여부=lambda df: np.where(df['부재일수'].isin(['0.5일(오전)', '0.5일(오후)']), 'Yes', 'No'))
# str 속성 사용하여 부재사유 이름 변경
.assign(부재사유=lambda df: df['부재사유'].str.slice(0, 2))
.loc[:, ['이름', '부재일수', '부재사유', '반차여부', '부재시작일', '부재종료일']]
)
leave_df

,이름,부재일수,부재사유,반차여부,부재시작일,부재종료일
0,조윤아,5일,연차,No,2020-01-20,2020-02-03
1,이준우,1일,연차,No,2020-01-02,2020-01-02
2,이준우,0.5일(오후),연차,Yes,2020-01-03,2020-01-03
3,이준우,0.5일(오후),연차,Yes,2020-01-08,2020-01-08
4,조윤아,1일,연차,No,2020-01-03,2020-01-03
...,...,...,...,...,...,...
5030,장준연,0.5일(오전),연차,Yes,2022-09-16,2022-09-16
5031,정예후,1일,연차,No,2022-09-19,2022-09-19
5032,임은현,0.5일(오후),연차,Yes,2022-09-16,2022-09-16
5033,강주윤,1일,연차,No,2022-09-23,2022-09-23


In [7]:
# 부재종료일이 잘못 입력되어있는 경우를 찾아 올바른 값으로 수정
leave_df[(leave_df['부재종료일'] - leave_df['부재시작일']).lt('0 days')]
leave_df.loc[1216, '부재종료일'] = pd.to_datetime('2021-01-08')
leave_df.loc[2057, '부재종료일'] = pd.to_datetime('2021-07-23')
leave_df.loc[4637, '부재종료일'] = pd.to_datetime('2021-06-21')

In [8]:
leave_df = (leave_df
.reset_index(drop=True)
.rename({'부재시작일':'Days1'}, axis=1)
)
leave_df

,이름,부재일수,부재사유,반차여부,Days1,부재종료일
0,조윤아,5일,연차,No,2020-01-20,2020-02-03
1,이준우,1일,연차,No,2020-01-02,2020-01-02
2,이준우,0.5일(오후),연차,Yes,2020-01-03,2020-01-03
3,이준우,0.5일(오후),연차,Yes,2020-01-08,2020-01-08
4,조윤아,1일,연차,No,2020-01-03,2020-01-03
...,...,...,...,...,...,...
5030,장준연,0.5일(오전),연차,Yes,2022-09-16,2022-09-16
5031,정예후,1일,연차,No,2022-09-19,2022-09-19
5032,임은현,0.5일(오후),연차,Yes,2022-09-16,2022-09-16
5033,강주윤,1일,연차,No,2022-09-23,2022-09-23


In [9]:
for i in range(2, int((leave_df['부재종료일'] - leave_df['Days1']).max() / '1 days') + 1):
    leave_df = (leave_df.assign(**{f'Days{i}':np.where(leave_df['부재종료일'].eq(leave_df[f'Days{i-1}']), leave_df[f'Days{i-1}'], leave_df[f'Days{i-1}'] + timedelta(days=1))})
    )
leave_df = (leave_df
.drop('부재종료일', axis=1))
leave_df

,이름,부재일수,부재사유,반차여부,Days1,Days2,Days3,Days4,Days5,Days6,Days7,Days8,Days9,Days10,Days11,Days12,Days13,Days14,Days15
0,조윤아,5일,연차,No,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03
1,이준우,1일,연차,No,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02,2020-01-02
2,이준우,0.5일(오후),연차,Yes,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03
3,이준우,0.5일(오후),연차,Yes,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08,2020-01-08
4,조윤아,1일,연차,No,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03,2020-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,장준연,0.5일(오전),연차,Yes,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16
5031,정예후,1일,연차,No,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19,2022-09-19
5032,임은현,0.5일(오후),연차,Yes,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16,2022-09-16
5033,강주윤,1일,연차,No,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23,2022-09-23


In [10]:
leave_df = (leave_df
.melt(id_vars=['이름', '부재일수', '부재사유', '반차여부'], value_name='부재일시')
.drop_duplicates(subset=['이름', '부재일수', '부재사유', '부재일시'])
.drop('variable', axis=1)
.sort_values('부재일시', ascending=True)
)
leave_df

,이름,부재일수,부재사유,반차여부,부재일시
1,이준우,1일,연차,No,2020-01-02
5,강주윤,2일,연차,No,2020-01-02
6,조도후,1일,연차,No,2020-01-02
7,장채진,1일,연차,No,2020-01-02
2,이준우,0.5일(오후),연차,Yes,2020-01-03
...,...,...,...,...,...
9948,장준연,5,연차,No,2022-11-15
14983,장준연,5,연차,No,2022-11-16
20018,장준연,5,연차,No,2022-11-17
25053,장준연,5,연차,No,2022-11-18


In [11]:
holiday_2020 = ['2020-01-01', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-03-01', '2020-04-15', 
               '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-09-30', '2020-10-01', '2020-10-02', 
               '2020-10-03', '2020-10-09', '2020-12-25']
holiday_2021 = ['2021-01-01', '2021-02-11', '2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19',
                '2021-06-06', '2021-08-15', '2021-08-16', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03',
                '2021-10-04', '2021-10-09', '2021-10-11', '2021-12-25']
holiday_2022 = ['2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-03-09', '2022-05-05', '2022-05-08',
                '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03',
                '2022-10-09', '2022-10-10', '2022-12-25']

In [12]:
leave_df = leave_df[~leave_df['부재일시'].dt.dayofweek.isin([5, 6]) & ~leave_df['부재일시'].isin(holiday_2020 + holiday_2021 + holiday_2022)]
leave_df

,이름,부재일수,부재사유,반차여부,부재일시
1,이준우,1일,연차,No,2020-01-02
5,강주윤,2일,연차,No,2020-01-02
6,조도후,1일,연차,No,2020-01-02
7,장채진,1일,연차,No,2020-01-02
2,이준우,0.5일(오후),연차,Yes,2020-01-03
...,...,...,...,...,...
9948,장준연,5,연차,No,2022-11-15
14983,장준연,5,연차,No,2022-11-16
20018,장준연,5,연차,No,2022-11-17
25053,장준연,5,연차,No,2022-11-18


In [13]:
leave_df = (leave_df
.assign(반차여부=np.where(leave_df['반차여부'] == 'No', 1, 0.5))
.assign(부재일수=leave_df['부재일수'].str.extract(r'(오전|오후)').fillna('전일'))
.reset_index(drop=True)
)
leave_df

,이름,부재일수,부재사유,반차여부,부재일시
0,이준우,전일,연차,1.0,2020-01-02
1,강주윤,전일,연차,1.0,2020-01-02
2,조도후,전일,연차,1.0,2020-01-02
3,장채진,전일,연차,1.0,2020-01-02
4,이준우,오후,연차,0.5,2020-01-03
...,...,...,...,...,...
7958,장준연,전일,연차,1.0,2022-11-15
7959,장준연,전일,연차,1.0,2022-11-16
7960,장준연,전일,연차,1.0,2022-11-17
7961,장준연,전일,연차,1.0,2022-11-18


In [15]:
total_df = (leave_df
.merge(info_df, left_on='이름', right_on='성명', how='inner')
.drop('성명', axis=1)
)
total_df

,이름,부재일수,부재사유,반차여부,부재일시,구분,소속,시작일,종료일,근속월수
0,이준우,전일,연차,1.0,2020-01-02,정규직,영업/마케팅팀,2019-08-05,2022-09-17,37.42
1,이준우,오후,연차,0.5,2020-01-03,정규직,영업/마케팅팀,2019-08-05,2022-09-17,37.42
2,이준우,오후,연차,0.5,2020-01-08,정규직,영업/마케팅팀,2019-08-05,2022-09-17,37.42
3,이준우,전일,연차,1.0,2020-02-20,정규직,영업/마케팅팀,2019-08-05,2022-09-17,37.42
4,이준우,오후,재택,0.5,2020-02-25,정규직,영업/마케팅팀,2019-08-05,2022-09-17,37.42
...,...,...,...,...,...,...,...,...,...,...
7958,임은현,전일,연차,1.0,2022-08-16,정규직,연구개발팀,2022-07-18,2022-09-17,2.01
7959,임은현,오후,연차,0.5,2022-08-26,정규직,연구개발팀,2022-07-18,2022-09-17,2.01
7960,임은현,오후,재택,0.5,2022-08-29,정규직,연구개발팀,2022-07-18,2022-09-17,2.01
7961,임은현,전일,재택,1.0,2022-09-06,정규직,연구개발팀,2022-07-18,2022-09-17,2.01


In [37]:
summary_df = (leave_df
.groupby([leave_df['부재일시'].dt.year.rename('연도'), '이름', '부재사유'])['반차여부']
.sum()
.unstack()
.rename_axis(None, axis=1)
.reset_index()
.fillna(0)
.merge(info_df, left_on='이름', right_on='성명', how='inner')
.drop('성명', axis=1)
.loc[:, ['연도', '이름', '구분', '소속', '시작일', '종료일', '근속월수', '연차', '재택', '출장']]
)
summary_df

,연도,이름,구분,소속,시작일,종료일,근속월수,연차,재택,출장
0,2020,강민서,정규직,영업/마케팅팀,2017-01-16,2022-09-17,68.02,28.5,24.0,0.0
1,2021,강민서,정규직,영업/마케팅팀,2017-01-16,2022-09-17,68.02,25.0,30.5,1.0
2,2022,강민서,정규직,영업/마케팅팀,2017-01-16,2022-09-17,68.02,16.5,23.5,0.0
3,2020,강서아,퇴사(정규직),영업/마케팅팀,2017-05-25,2022-09-17,63.78,16.0,1.0,0.0
4,2021,강서아,퇴사(정규직),영업/마케팅팀,2017-05-25,2022-09-17,63.78,14.0,22.5,0.0
...,...,...,...,...,...,...,...,...,...,...
158,2022,장서태,정규직,영업/마케팅팀,2021-11-22,2022-09-17,9.83,8.5,15.0,0.0
159,2022,장준연,정규직,연구개발팀,2022-04-11,2022-09-17,5.23,11.5,7.0,0.0
160,2022,장지후,정규직,연구개발팀,2021-12-27,2022-09-17,8.68,15.0,47.5,0.0
161,2022,장하태,인턴,연구개발팀,2022-07-11,2022-09-17,2.24,5.0,7.0,0.0
